# Template for task: Create GenAI Application to Gather User Requirement

In [1]:
import time
import os
from openai import OpenAI
 
client = OpenAI()

In [2]:
def read_specification():
    specification = {}
 
    system_overview_file_path = 'specification/system_overview.txt'
    requirement_file_path = 'specification/requirement.txt'
    customer_file_path = 'specification/customer.txt'
    data_to_track_file_path = 'specification/data_to_track.txt'
    technical_requirement_file_path = 'specification/technical_requirement.txt'
 
    # system overview
    if os.path.exists(system_overview_file_path):
        with open(system_overview_file_path, 'r') as file:
            specification['system_overview'] = file.read()
    else:
        raise FileNotFoundError(f"The file '{system_overview_file_path}' does not exist.")
 
    # requirement
    if os.path.exists(requirement_file_path):
        with open(requirement_file_path, 'r') as file:
            specification['requirement'] = file.read()
    else:
        raise FileNotFoundError(f"The file '{requirement_file_path}' does not exist.")
    
    # customer
    if os.path.exists(customer_file_path):
        with open(customer_file_path, 'r') as file:
            specification['customer'] = file.read()
    else:
        raise FileNotFoundError(f"The file '{customer_file_path}' does not exist.")
    
    # data to track
    if os.path.exists(data_to_track_file_path):
        with open(data_to_track_file_path, 'r') as file:
            specification['data_to_track'] = file.read()
    else:
        raise FileNotFoundError(f"The file '{data_to_track_file_path}' does not exist.")
    
    # technical requirement
    if os.path.exists(technical_requirement_file_path):
        with open(technical_requirement_file_path, 'r') as file:
            specification['technical_requirement'] = file.read()
    else:
        raise FileNotFoundError(f"The file '{technical_requirement_file_path}' does not exist.")
    
    
    return specification

In [3]:
def create_database_design_prompt(specification):
    parts = []
    parts.append("Design a relational database based on this specification.")
    
    parts.append("<system-overview>")
    parts.append(specification['system_overview'])
    parts.append("</system-overview>")
    
    parts.append("<requirement>")
    parts.append(specification['requirement'])
    parts.append("</requirement>")
    
    parts.append("<customer>")
    parts.append(specification['customer'])
    parts.append("</customer>")
    
    parts.append("<data-to-track>")
    parts.append(specification['data_to_track'])
    parts.append("</data-to-track>")
    
    parts.append("<technical-requirement>")
    parts.append(specification['technical_requirement'])
    parts.append("</technical-requirement>")
    
    parts.append("----------")
    
    parts.append("Your output must be in the following JSON format.")
    parts.append("In the JSON value, don't use any line break (e.g. \\n).")
    parts.append("All table names, column names, relationship names, and query names must be in lower_snake_case.")
    parts.append("Table name must be in plural noun.")
    parts.append("Column name must be in singular noun.")
    parts.append("Make the tables.columns[n].constraints as null if your design does not has any constraint")
    parts.append("On FOREIGN KEY constraints, don't use CASCADE, SET NULL or SET DEFAULT")
    parts.append("""
    {
      "table_designs":[
        {
          "name":"sequence_table_name. The 'sequence' is a sequence indicates the order of the table to be created, in 2 digit numeric. Thus, a table which mas foreign key constraint must be created after the main table to be referred to.",
          "columns":[
            {
              "column_name":"column name",
              "data_type":"SQL data type",
              "constraints":[
                "constraint 1",
                "constraint 2"
              ],
              "sample_data":"the data sample for this column",
              "description":"column's description"
            }
          ],
          "sql":"SQL Data Definition Language to create this table. The SQL must contain the proper IF NOT EXISTS keyword. Create column constraint (e.g. not null). Create relationships / foreign keys constraint."
        }
      ],
      "query_designs":[
        {
          "name":"query_name. For each requirement on <data-to-track>, write the query based on the database that you designed",
          "sql":"the SQL query based on the database that you designed"
        }
      ]
    }
    """)
    
    return '\n'.join(parts)

In [4]:
def call_genai(prompt):
    for nr_retries in range(1, 4):
        try:
            response = client.chat.completions.create(
                model='gpt-4o-mini',
                messages=[
                        {
                            'role':'system', 
                            'content':'You are an SQL database designer tasked to design a database using best practices in relational database design.'
                        },
                        {
                            'role':'user', 
                            'content':prompt
                        }
                    ]
                )
            return response.choices[0].message.content
        except:
            time.sleep(5)
    raise Exception('Cannot query OpenAI model!')

In [5]:
# Read the specification details and generate the database design prompt using OpenAI.
# Write the response to the output file
specification = read_specification()    
prompt = create_database_design_prompt(specification)
 
response = call_genai(prompt)
 
output_file_path = 'genai_design/database_raw_design.json'
 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
 
with open(output_file_path, 'w') as file:
    file.write(response)
print(f"The database design has been written to '{output_file_path}'.")

Exception: Cannot query OpenAI model!